In [1]:
import cv2
import math
import time
import random
import pygame
import mediapipe as mp

pygame 2.5.1 (SDL 2.28.2, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Block:
    def __init__(self,x,y,w,h,c):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.c = c

In [3]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
hands = mp_hands.Hands()

pygame.init()

width, height = 450, 550
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("FALLINGBLOCKS")

scroll_y = 0
bg = pygame.image.load("assets/bg.jpg")
scaled_bg = pygame.transform.scale(bg, (bg.get_width() // 4, bg.get_height() // 4))

block_width = 200
block_height = 30
block_x = width // 2 - block_width // 2
block_y = height - 30

main_block = Block(block_x,block_y,block_width,block_height,(0,0,0))
main_block_draw = pygame.Rect(main_block.x,main_block.y,main_block.w,main_block.h)

prev_width = block_width
prev_y = block_y
prev_x = block_x

blocks = []
blocks.append(main_block)

start = False
moving = False
fall = False

speed = 15
direction = 1
prev_direction = direction

remaining = 0

game_over = False
game_over_font = pygame.font.SysFont("Unique", 100) 
game_over_text = game_over_font.render("GAME OVER!", True, (255,255,255)) 

score = 0
score_font = pygame.font.SysFont("Unique", 100) 
score_text = score_font.render(f"SCORE: {score}", True, (255,255,255)) 

running = True 

while running:
    state, frame = cap.read()
    
    if state:
        frame = cv2.flip(frame, 1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                tip = [hand_landmarks.landmark[i].y for i in [8, 12, 16, 20]]
                dip = [hand_landmarks.landmark[i - 1].y for i in [8, 12, 16, 20]]

                opened = any(tip[i] < dip[i] for i in range(4))

                if not opened and moving:
                    fall = True


        screen.blit(scaled_bg,(0,0))

        if not game_over:

            if not moving:
                new_block = Block(prev_x,prev_y-60,prev_width,block_height,((0,0,0)))
                moving = True
                direction = prev_direction

            if new_block.y <= height//2:
                scroll_y += 30
                prev_y += 30
                for block in blocks:
                    block.y += 30
                new_block.y += 30
                blocks.pop(0)                    
                background_y = -scroll_y % scaled_bg.get_height()
                screen.blit(scaled_bg, (0, background_y))
                screen.blit(scaled_bg, (0, background_y - scaled_bg.get_height()))
                prev_x -= 5

            elif moving and not fall:
                if new_block.x <= 0:  
                    direction = 1
                elif new_block.x + new_block.w >= width:
                    direction = -1
                new_block.x += speed * direction
                prev_direction = direction

            if fall:

                if any(new_block.x + new_block.w <= block.x or new_block.x >= block.x + block.w for block in blocks):
                    new_block.y += 20
                    if new_block.y >= height:
                        game_over = True

                direction = 0
                new_block.y += 5

                if new_block.y == prev_y-30:

                    temp = new_block.y
                    new_block.y = prev_y - 30

                    if new_block.x > prev_x:
                        remaining = new_block.x - prev_x
                        new_block.w -= remaining
                        score += 1
                    elif new_block.x < prev_x:
                        remaining = prev_x - new_block.x
                        new_block.w -= remaining
                        new_block.x = prev_x
                        score += 1
                    else:
                        score += 10

                    prev_y = temp
                    prev_x = new_block.x
                    prev_width = new_block.w

                    blocks.append(new_block)
                    moving = False
                    fall = False


        moving_draw = pygame.Rect(new_block.x,new_block.y,new_block.w,new_block.h)
        pygame.draw.rect(screen,new_block.c,moving_draw)

        for block in blocks:
            block_draw = pygame.Rect(block.x,block.y,block.w,block.h)
            pygame.draw.rect(screen,block.c,block_draw)
            
        if new_block.w == 0:
            game_over = True    
        
        if game_over:
            game_over_rect = game_over_text.get_rect(center=(width // 2, height // 2))
            screen.blit(game_over_text, game_over_rect)

        score_text = score_font.render(f"SCORE: {score}", True, (255,255,255))  
        score_rect = score_text.get_rect(center=(width // 2,50))
        screen.blit(score_text, score_rect)
     
        pygame.display.flip()     
                        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.KEYDOWN :
                if event.key == pygame.K_r and game_over:
                    block_width = 200
                    block_height = 30
                    block_x = width // 2 - block_width // 2
                    block_y = height - 30
                    prev_width = block_width
                    prev_y = block_y
                    prev_x = block_x
                    blocks = []
                    main_block = Block(block_x,block_y,block_width,block_height,(0,0,0))
                    main_block_draw = pygame.Rect(main_block.x,main_block.y,main_block.w,main_block.h)
                    blocks.append(main_block)
                    start = False
                    moving = False
                    fall = False
                    speed = 15
                    direction = 1
                    prev_direction = direction
                    remaining = 0
                    game_over = False
                    score = 0
                    scroll_y = 0                

    cv2.imshow("STOP", frame)

    cv2.waitKey(1)

cap.release()
cv2.destroyAllWindows()
pygame.quit()
